In [24]:
import pandas as pd
import datetime 

In [25]:
AI_FP = pd.read_excel('Sample.xlsx', sheetname='AI FP Equity', index_col='Timestamp')
AI_FP.head()

,Open,Close,Price,Volume
Timestamp,,,,
2018-01-02 08:00:00,105.00,105.05,105.05,15086
2018-01-02 08:01:00,105.10,105.00,105.00,2073
2018-01-02 08:02:00,104.90,104.70,104.70,3050
2018-01-02 08:03:00,104.65,104.80,104.80,1496
2018-01-02 08:04:00,104.80,104.70,104.70,3341


In [26]:
ALV_GR = pd.read_excel('Sample.xlsx', sheetname='ALV GR Equity', index_col='Timestamp')
ALV_GR.head()

,Open,Close,Price,Volume
Timestamp,,,,
2018-01-02 07:00:00,192.010,192.200,192.200,2103
2018-01-02 07:01:00,192.300,192.180,192.180,1832
2018-01-02 07:03:00,191.945,191.945,191.945,500
2018-01-02 07:04:00,191.997,192.035,192.035,1300
2018-01-02 07:05:00,192.044,192.078,192.078,500


In [28]:
ALV_GR.index = ALV_GR.index + pd.Timedelta(hours=1)
ALV_GR.index

In [30]:
print(ALV_GR.shape)
print(AI_FP.shape)

(19781, 4)
(14758, 4)


In [56]:
# generate the index manually. Otherwise standard resampling methods would include all the 
# hours of the day! We do care only about trading hours and Business Days (freq='B')
index_date = pd.date_range('2018-01-02', '2018-02-15', freq='B')
index_date = pd.Series(index_date)
index_time = pd.date_range('08:00:00', '16:40:00', freq='1min')
index_time = pd.Series(index_time.time)

index = index_date.apply(
    lambda d: index_time.apply(
        lambda t: datetime.datetime.combine(d, t)
        )
    ).unstack().sort_values().reset_index(drop=True)

print(index.shape)

(17193,)


In [57]:
# reindex the Series with the newly generated index
ALV_GR = ALV_GR.reindex(index=index)

# interpolate to get the values for the NaNs
ALV_GR_interp = ALV_GR.interpolate(method='linear')
ALV_GR_interp.tail()

,Open,Close,Price,Volume
2018-02-15 16:36:00,189.68,189.56,189.56,4032.0
2018-02-15 16:37:00,189.60,189.48,189.48,2224.0
2018-02-15 16:38:00,189.50,189.52,189.52,1531.0
2018-02-15 16:39:00,189.44,189.32,189.32,3591.0
2018-02-15 16:40:00,189.34,189.44,189.44,359.0


In [58]:
# reindex the Series with the newly generated index
AI_FP = AI_FP.reindex(index=index)

# interpolate to get the values for the NaNs
AI_FP_interp = AI_FP.interpolate(method='linear')
AI_FP_interp.tail()

,Open,Close,Price,Volume
2018-02-15 16:36:00,101.0,101.0,101.0,520930.0
2018-02-15 16:37:00,101.0,101.0,101.0,520930.0
2018-02-15 16:38:00,101.0,101.0,101.0,520930.0
2018-02-15 16:39:00,101.0,101.0,101.0,520930.0
2018-02-15 16:40:00,101.0,101.0,101.0,520930.0


In [60]:
print(ALV_GR_interp.shape)
print(AI_FP_interp.shape)

(17193, 4)
(17193, 4)


In [62]:
ALV_GR_interp.join(AI_FP_interp, lsuffix='ALV', rsuffix='AI')

,OpenALV,CloseALV,PriceALV,VolumeALV,OpenAI,CloseAI,PriceAI,VolumeAI
2018-01-02 08:00:00,192.010000,192.20000,192.20000,2103.000000,105.00,105.05,105.05,15086.0
2018-01-02 08:01:00,192.300000,192.18000,192.18000,1832.000000,105.10,105.00,105.00,2073.0
2018-01-02 08:02:00,192.122500,192.06250,192.06250,1166.000000,104.90,104.70,104.70,3050.0
2018-01-02 08:03:00,191.945000,191.94500,191.94500,500.000000,104.65,104.80,104.80,1496.0
2018-01-02 08:04:00,191.997000,192.03500,192.03500,1300.000000,104.80,104.70,104.70,3341.0
2018-01-02 08:05:00,192.044000,192.07800,192.07800,500.000000,104.70,104.60,104.60,1654.0
2018-01-02 08:06:00,192.086000,192.22400,192.22400,400.000000,104.60,104.50,104.50,4394.0
2018-01-02 08:07:00,192.500000,192.26000,192.26000,698.000000,104.45,104.35,104.35,7482.0
2018-01-02 08:08:00,192.315500,192.19550,192.19550,499.000000,104.25,104.10,104.10,5873.0
2018-01-02 08:09:00,192.131000,192.13100,192.13100,300.000000,104.10,104.20,104.20,3673.0


# Load Bloomberg data from Excel 

In [161]:
data4 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet4', index_col='Timestamp')
data5 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet5', index_col='Timestamp')
data6 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet6', index_col='Timestamp')
data7 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet7', index_col='Timestamp')
data8 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet8', index_col='Timestamp')
data9 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet9', index_col='Timestamp')
data10 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet10', index_col='Timestamp')
data11 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet11', index_col='Timestamp')
data12 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet12', index_col='Timestamp')
data13 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet13', index_col='Timestamp')
data14 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet14', index_col='Timestamp')
data15 = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet15', index_col='Timestamp')

In [162]:
# align the timezone
data5.index = data5.index + pd.Timedelta(hours=1)
data13.index = data13.index + pd.Timedelta(hours=1)
data14.index = data14.index + pd.Timedelta(hours=1)
data15.index = data15.index + pd.Timedelta(hours=1)

In [163]:
# more data
data = dict()

j = 0
for i in range(0, 107, 3):
    data[j] = pd.read_excel('BloombergDataCleaned.xlsm', sheetname='Sheet0', parse_cols=[i,i+1], index_col=0)
    # drop nan
    data[j].dropna(inplace=True)
    # reindex to align the timezone
    if data[j].index[0].time().hour == 7:
        data[j].index = data[j].index + pd.Timedelta(hours=1)
    j = j+1

In [ ]:
'''# reindex to align the timezone
data[3].index = data[3].index + pd.Timedelta(hours=1)
data[5].index = data[5].index + pd.Timedelta(hours=1)
data[6].index = data[6].index + pd.Timedelta(hours=1)
data[10].index = data[10].index + pd.Timedelta(hours=1)
data[18].index = data[18].index + pd.Timedelta(hours=1)
data[22].index = data[22].index + pd.Timedelta(hours=1)
data[25].index = data[25].index + pd.Timedelta(hours=1)
'''
# you could also do it dynamically using 
''' 
for i in range(0, 36):
    if data[i].index[0].time().hour == 7:
        data[i].index = data[i].index + pd.Timedelta(hours=1)
'''

In [164]:
# generate the index manually. Otherwise standard resampling methods would include all the 
# hours of the day! We do care only about trading hours and Business Days (freq='B')
index_date = pd.date_range('2018-01-02', '2018-02-15', freq='B')
index_date = pd.Series(index_date)
index_time = pd.date_range('08:00:00', '16:40:00', freq='1min')
index_time = pd.Series(index_time.time)

index = index_date.apply(
    lambda d: index_time.apply(
        lambda t: datetime.datetime.combine(d, t)
        )
    ).unstack().sort_values().reset_index(drop=True)

print(index.shape)

(17193,)


In [165]:
# reindex the Series with the newly generated index
data4 = data4.reindex(index=index)
data5 = data5.reindex(index=index)
data6 = data6.reindex(index=index)
data7 = data7.reindex(index=index)
data8 = data8.reindex(index=index)
data9 = data9.reindex(index=index)
data10 = data10.reindex(index=index)
data11 = data11.reindex(index=index)
data12 = data12.reindex(index=index)
data13 = data13.reindex(index=index)
data14 = data14.reindex(index=index)
data15 = data15.reindex(index=index)

# interpolate to get the values for the NaNs
data4 = data4.interpolate(method='linear')
data5 = data5.interpolate(method='linear')
data6 = data6.interpolate(method='linear')
data7 = data7.interpolate(method='linear')
data8 = data8.interpolate(method='linear')
data9 = data9.interpolate(method='linear')
data10 = data10.interpolate(method='linear')
data11 = data11.interpolate(method='linear')
data12 = data12.interpolate(method='linear')
data13 = data13.interpolate(method='linear')
data14 = data14.interpolate(method='linear')
data15 = data15.interpolate(method='linear')

In [ ]:
# move everything into a single dictionary
data[36] = data4
data[37] = data5
data[38] = data6
data[39] = data7
data[40] = data8
data[41] = data9
data[42] = data10
data[43] = data11
data[44] = data12
data[45] = data13
data[46] = data14
data[47] = data15

In [228]:
# create an empty df indexed like my data
data_open = pd.DataFrame(index=index)

# join all the dictionary items into one single df
for k in data.keys():
    data_open = data_open.join(data[k])
print(data_open.shape)

(17193, 48)


In [229]:
# save the data into a csv file 
data_open.to_csv('closing_prices_cleaned.csv')

# End of data preparation 